# Train Faster-RCNN

In [1]:
import pprint
import numpy as np
import pdb
import sys
import os.path
import glob
import shutil
import tensorflow as tf

this_dir = os.chdir("/root/faster_rcnn")

from lib.fast_rcnn.train import get_training_roidb, train_net
from lib.fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir, get_log_dir
from lib.datasets.factory import get_imdb
from lib.networks.factory import get_network

kittivoc_train
kittivoc_val
kittivoc_trainval
kittivoc_test
kittivoc_train
kittivoc_val
kittivoc_trainval
kittivoc_test
kittivoc_train
kittivoc_val
kittivoc_trainval
kittivoc_test
nthu_71
nthu_370


## Set up Training Schedule

### Specify and load configuration:

In [2]:
cfg_file = "experiments/cfgs/faster_rcnn_end2end_wider.yml"
network_name = "VGGnet_featconcat_train"
pretrained_model = "data/pretrain_model/VGG_imagenet.npy"
imdb_name = "voc_2007_trainval"
randomize = True
restore = True
ITERS = 210000

cfg_from_file(cfg_file)

# User-defined additional configuration.
cfg.GPU_ID = 0
cfg.ANCHOR_SCALES = [4,8,16,32]

print('Using config:')
pprint.pprint(cfg)

Using config:
{'ANCHOR_SCALES': [4, 8, 16, 32],
 'DATA_DIR': '/root/faster_rcnn/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'faster_rcnn_voc_vgg',
 'GPU_ID': 0,
 'IS_EXTRAPOLATING': True,
 'IS_MULTISCALE': False,
 'IS_RPN': True,
 'LOG_DIR': 'faster_rcnn_voc_vgg',
 'MATLAB': 'matlab',
 'MODELS_DIR': '/root/faster_rcnn/models/pascal_voc',
 'NCLASSES': 2,
 'NET_NAME': 'VGGnet',
 'PIXEL_MEANS': array([[[ 102.9801,  115.9465,  122.7717]]]),
 'REGION_PROPOSAL': 'RPN',
 'RNG_SEED': 3,
 'ROOT_DIR': '/root/faster_rcnn',
 'SUBCLS_NAME': 'voxel_exemplars',
 'TEST': {'BBOX_REG': True,
          'HARDNEG_CONF': 0.8,
          'HARDNEG_OVERLAP': 0.5,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'selective_search',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 300,
          'RPN_PRE_NMS_TOP_N': 6000,
          'SCALES': [600],
          'SVM': False},
 'TRAIN': {'ASPECTS': [1]

In [ ]:
# Specify number of folds to train
N_FOLDS = 1
# Identify experiment number
N_ID = 1

### Train Model:

ID | Iterations | Anchor Scales | FeatConcat | HardNeg | Multiscale
--- | --- | --- | --- | --- | ---
1 | 210k | 4 | Y | N | N
2 | 210k | 4 | Y | Y | N

In [ ]:
if randomize:
    # Fix the random seeds (numpy and caffe) for reproducibility
    np.random.seed(cfg.RNG_SEED)
    
for fold in range(1,N_FOLDS+1):

    imdb = get_imdb(imdb_name)
    print('Loaded dataset `{:s}` for training'.format(imdb.name))
    roidb = get_training_roidb(imdb)
    
    # Define output directory
    output_dir = get_output_dir(imdb, None)
    output_dir = os.path.join(output_dir,"ID_{0:d}".format(N_ID))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    print('Output will be saved to `{:s}`'.format(output_dir))
    
    # Define log directory
    log_dir = get_log_dir(imdb)
    print('Logs will be saved to `{:s}`'.format(log_dir))
    
    # Reset checkpoint file
    #if restore:
    #    shutil.copy(os.path.join(output_dir,"checkpoint.orig"),os.path.join(output_dir,"checkpoint"))
    
    # Define graph
    network = get_network(network_name)
    print('Use network `{:s}` in training'.format(network_name))

    train_net(network, imdb, roidb,
              output_dir=output_dir,
              log_dir=log_dir,
              pretrained_model=pretrained_model,
              max_iters=ITERS,
              restore=bool(int(restore)))
    
    # Graph must be reset before we restart training for the next fold.
    tf.reset_default_graph()
    
    # Define permanent save directory.
    save_dir = os.path.join(output_dir,"Main_{0:d}".format(fold))
    if os.path.exists(save_dir):
        shutil.rmtree(save_dir)
    os.mkdir(save_dir)
    
    # Move output to save directory to prevent overwriting.
    for ckpt_path in glob.glob(str(os.path.join(output_dir,'*{0:d}.ckpt.*'.format(ITERS)))):
            ckpt_file = os.path.basename(ckpt_path)
            shutil.move(ckpt_path,os.path.join(save_dir,ckpt_file))
            print('Moved checkpoint file {0:s} to {1:s}'.format(ckpt_file,save_dir))

<bound method pascal_voc.default_roidb of <lib.datasets.pascal_voc.pascal_voc object at 0x7f59178df650>>
Loaded dataset `voc_2007_trainval` for training
Appending horizontally-flipped training examples...
voc_2007_trainval gt roidb loaded from /root/faster_rcnn/data/cache/voc_2007_trainval_gt_roidb.pkl
done
Preparing training data...
done
Output will be saved to `/root/faster_rcnn/output/faster_rcnn_voc_vgg/voc_2007_trainval/ID_1`
Logs will be saved to `/root/faster_rcnn/logs/faster_rcnn_voc_vgg/voc_2007_trainval/2017-06-21-11-34-47`
Tensor("data:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("conv5_3/Relu:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_conv/3x3/Relu:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_conv/3x3/Relu:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_cls_score/BiasAdd:0", shape=(?, ?, ?, 24), dtype=float32)
Tensor("gt_boxes:0", shape=(?, 5), dtype=float32)
Tensor("gt_ishard:0", shape=(?,), dtype=int32)
Tensor("dontcare_areas:0", shape=(?, 4), dtyp

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Restoring from /root/faster_rcnn/output/faster_rcnn_voc_vgg/voc_2007_trainval/ID_1/VGGnet_fast_rcnn_iter_110000.ckpt...
INFO:tensorflow:Restoring parameters from /root/faster_rcnn/output/faster_rcnn_voc_vgg/voc_2007_trainval/ID_1/VGGnet_fast_rcnn_iter_110000.ckpt
done
iter: 110000 / 210000, total loss: 0.1913, rpn_loss_cls: 0.0107, rpn_loss_box: 0.1242, loss_cls: 0.0281, loss_box: 0.0283, lr: 0.000100
speed: 1.764s / iter
image: 004164.jpg iter: 110010 / 210000, total loss: 1.0153, rpn_loss_cls: 0.0496, rpn_loss_box: 0.2249, loss_cls: 0.1450, loss_box: 0.5958, lr: 0.000100
speed: 1.311s / iter
image: 014522.jpg iter: 110020 / 210000, total loss: 0.3296, rpn_loss_cls: 0.0101, rpn_loss_box: 0.1018, loss_cls: 0.0154, loss_box: 0.2022, lr: 0.000100
speed: 0.263s / iter
image: 005244.jpg iter: 110030 / 210000, total loss: 0.8060, rpn_loss_cls: 0.0492, rpn_loss_box: 0.0466, loss_cls: 0.0470, loss_box: 0.6633, lr: 0.000100
speed: 1.013s / iter
image: 005369.jpg iter: 110040 / 210000, total lo